In [1]:
base_df = 'kaggle/'
stock_df = 'kaggle/stocks/'
etfs_df = 'kaggle/etfs/'

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
valid_data = pd.read_csv(stock_df+"A.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'kaggle/stocks/A.csv'

In [81]:
valid_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300
1,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100
2,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800
3,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600
4,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200


In [99]:
def datasetConstruct(arq):
    a_data = pd.read_csv(os.path.join(stock_df,arq))
    if a_data.isna().sum().sum()>0:
        a_data = pd.DataFrame()
        return a_data
    
    a_data['Date'] = pd.to_datetime(a_data['Date'])
    a_data['Day'] = a_data['Date'].dt.day 
    a_data['Month'] = a_data['Date'].dt.month
    a_data['Year'] = a_data['Date'].dt.year
    month_close = a_data.groupby(['Year','Month']).agg(
        Media_Close_Average=('Close','mean'),
        Media_Volume_Average=('Volume','mean')
    ).reset_index()

    month_close = month_close.sort_values(by=['Year', 'Month']).reset_index(drop=True)

    # Calcular a coluna target
    targets = []
    
    # Iterar sobre os índices, exceto o último
    for i in range(len(month_close) - 1):
        # Define target como 1 se Media_Average do próximo mês for maior, caso contrário 0
        targets.append(1 if month_close['Media_Close_Average'].iloc[i] < month_close['Media_Close_Average'].iloc[i + 1] else 0)
    
    # Adiciona um 0 para o último mês do dataset, pois não há mês seguinte para comparação
    month_close = month_close.iloc[:-1].copy()
    
    # Atribuir a lista de targets ao dataframe
    month_close['Target'] = targets
    return month_close


In [100]:
data = pd.DataFrame()
lista = os.listdir(stock_df)
for arq in lista:
    dataConstruct = datasetConstruct(arq=arq)
    data = pd.concat([data,dataConstruct])
data = data.reset_index(drop=True)
data

,Year,Month,Media_Close_Average,Media_Volume_Average,Target
0,2015,10,4.577500,6624.000000,1.0
1,2015,11,4.590000,1138.350000,1.0
2,2015,12,4.734091,204.545455,0.0
3,2016,1,4.710526,251.473684,0.0
4,2016,2,4.202778,493.055556,1.0
...,...,...,...,...,...
1068253,2019,11,42.190751,25.000000,0.0
1068254,2019,12,40.918286,14.285714,0.0
1068255,2020,1,40.630524,19.047619,1.0
1068256,2020,2,42.012421,189.473684,1.0


In [101]:
data = data.dropna()

In [102]:
data['Media_Close_Average'].describe().T

count    1.068258e+06
mean     3.328574e+06
std      5.483902e+08
min      1.500000e-04
25%      6.694000e+00
50%      1.461591e+01
75%      2.951206e+01
max      1.461348e+11
Name: Media_Close_Average, dtype: float64

In [103]:
X,y = data[['Media_Close_Average','Media_Volume_Average']],data['Target']

In [104]:
X.isnull().sum()

Media_Close_Average     0
Media_Volume_Average    0
dtype: int64

In [105]:
y.isnull().sum()

np.int64(0)

In [106]:
from sklearn import preprocessing

X = np.asarray(X).astype('float32')
# X = np.nan_to_num(X,nan=0)

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X)
X = pd.DataFrame(x_scaled)

In [107]:
X.isnull().sum()

,0,1
0,3.132279e-11,2.976045e-06
1,3.140833e-11,5.114404e-07
2,3.239434e-11,9.189863e-08
3,3.223309e-11,1.129826e-07
4,2.875857e-11,2.215211e-07
...,...,...
1068253,2.887102e-10,1.123206e-08
1068254,2.800027e-10,6.418317e-09
1068255,2.780335e-10,8.557756e-09
1068256,2.874898e-10,8.512715e-08


In [108]:
y = np.asarray(y).astype('int')
y

array([1, 1, 0, ..., 1, 1, 1])

In [130]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3)

print('Shape de x_train', x_train.shape)
print('Shape de x_val', x_val.shape)
print('Shape de x_test', x_test.shape)

print('Shape de y_train', y_train.shape)
print('Shape de y_val', y_val.shape)
print('Shape de y_test', y_test.shape)

Shape de x_train (598224, 2)
Shape de x_val (256382, 2)
Shape de x_test (213652, 2)
Shape de y_train (598224,)
Shape de y_val (256382,)
Shape de y_test (213652,)


In [131]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam, RMSprop, Adadelta, Adamax
from tensorflow.keras.callbacks import EarlyStopping

Adam = Adam(
    learning_rate=0.1
)

model = Sequential([
    Input(shape=(x_train.shape[1],)),
    Dense(4000, activation = 'relu'),
    # BatchNormalization(),
    Dense(3000, activation = 'relu'),
    # BatchNormalization(),
    Dense(2000, activation = 'relu'),
    # BatchNormalization(),
    Dense(1000, activation = 'relu'),
    # BatchNormalization(),
    Dense(750, activation = 'relu'),
    # BatchNormalization(),
    Dense(500, activation = 'relu'),
    # BatchNormalization(),
    Dense(250, activation = 'relu'),
    # BatchNormalization(),
    Dense(125, activation = 'relu'),
    # BatchNormalization(),
    Dense(64, activation = 'selu'),
    # BatchNormalization(),
    Dense(32, activation = 'relu'),
    # BatchNormalization(),
    Dense(16, activation = 'selu'),
    # BatchNormalization(),
    Dense(8, activation = 'relu'),
    # BatchNormalization(),
    Dense(4, activation = 'selu'),
    # BatchNormalization(),
    Dense(2, activation = 'softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adamax(), metrics=['accuracy'])

In [132]:
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_98 (Dense)                │ (None, 4000)           │        12,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_99 (Dense)                │ (None, 3000)           │    12,003,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_100 (Dense)               │ (None, 2000)           │     6,002,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_101 (Dense)               │ (None, 1000)           │     2,001,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_102 (Dense)               │ (None, 750)            │       750,750 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_103 (Dense)               │ (None, 500)            │       375,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_104 (Dense)               │ (None, 250)            │       125,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_105 (Dense)               │ (None, 125)            │        31,375 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_106 (Dense)               │ (None, 64)             │         8,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_107 (Dense)               │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_108 (Dense)               │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_109 (Dense)               │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_110 (Dense)               │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_111 (Dense)               │ (None, 2)              │            10 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,311,729 (81.30 MB)

 Trainable params: 21,311,729 (81.30 MB)

 Non-trainable params: 0 (0.00 B)

In [133]:
batch_size = 32
epochs = 300
callback = EarlyStopping(
    monitor = 'accuracy',
    min_delta = 0.01,
    patience = 5,
    verbose = 1
)
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                   callbacks = callback)

Epoch 1/300
18695/18695 ━━━━━━━━━━━━━━━━━━━━ 183s 10ms/step - accuracy: 0.5205 - loss: 0.6924 - val_accuracy: 0.5228 - val_loss: 0.6921
Epoch 2/300
18695/18695 ━━━━━━━━━━━━━━━━━━━━ 173s 9ms/step - accuracy: 0.5209 - loss: 0.6923 - val_accuracy: 0.5228 - val_loss: 0.6922
Epoch 3/300
18695/18695 ━━━━━━━━━━━━━━━━━━━━ 174s 9ms/step - accuracy: 0.5206 - loss: 0.6923 - val_accuracy: 0.5228 - val_loss: 0.6923
Epoch 4/300
18695/18695 ━━━━━━━━━━━━━━━━━━━━ 173s 9ms/step - accuracy: 0.5227 - loss: 0.6922 - val_accuracy: 0.5228 - val_loss: 0.6925
Epoch 5/300
18695/18695 ━━━━━━━━━━━━━━━━━━━━ 174s 9ms/step - accuracy: 0.5211 - loss: 0.6923 - val_accuracy: 0.5228 - val_loss: 0.6922
Epoch 6/300
18695/18695 ━━━━━━━━━━━━━━━━━━━━ 173s 9ms/step - accuracy: 0.5210 - loss: 0.6923 - val_accuracy: 0.5228 - val_loss: 0.6922
Epoch 6: early stopping
